In [1]:
######Initializing all the python packages for building dashboard

###Refer to the official Dash website to understand the syntax better

import datetime

from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import dash_table_experiments as dt
from dash import Dash, dash_table
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
#app = JupyterDash(external_stylesheets=external_stylesheets)

app = JupyterDash()  #####This is to run the code through Jupyter lab

####Function to print the obtained result python dataframe as a table through HTML
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

def img_conv():
    import sys
    from PIL import Image
    from io import BytesIO
    import base64
    
    content_type, content_string = aa[0].split(',')
    decoded = base64.b64decode(content_string)

    stream = BytesIO(decoded)

    image = Image.open(stream).convert("RGBA")
    stream.close()
    image.save("img.png")

    import cv2
    img = cv2.imread("img.png")   ####Reading the uploaded image
    img_text(img)

def img_text(image):

    import os

    import pandas as pd
    from matplotlib import pyplot as plt
    from easyocr import Reader ###Pretrained text extraction model from an AI company JaidedAI

    #from IPython.display import clear_output
    #import time 

    import re
    import nltk #####NLP Packages to do text analysis

    langs = ['en']   
    reader = Reader(langs, gpu=0)
    
    print("1")
    ###Reading text from Image ####
    def plt_imshow(title, image):
        # convert the image frame BGR to RGB color space and display it
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.title(title)
        plt.grid(False)
        plt.show()

    def cleanup_text(text):
        # strip out non-ASCII text so we can draw the text on the image
        # using OpenCV
        return "".join([c if ord(c) < 128 else "" for c in text]).strip()
    img = image
    img_name = 'img.png'

    pix = 0
    cnt = 0

    img_nme = []
    pix_perc = []

    txt_pix = []
    img_pix = []
    per_pix = []
    per_perc = []

    width = []
    height = []

    contr =[]
    contr_r =[]
    pers_cnt =[]
    txtt = []
    na_i = []

    txt = []
    shape = (img.shape[0] * img.shape[1])

    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast_r = img_grey.std()  ###to calculate the contrast of the image

    print("2")
    results = reader.readtext(img)

    print("4")
    for (bbox, text, prob) in results:
        #if "unacademy" in text:

        (tl, tr, br, bl) = bbox
        tl = (int(tl[0]), int(tl[1]))
        tr = (int(tr[0]), int(tr[1]))
        br = (int(br[0]), int(br[1]))
        bl = (int(bl[0]), int(bl[1]))

        wid = br[0] - tl[0]
        hgt = br[1] - tl[1]

        pix1 = wid * hgt
        pix = pix + pix1

        text = cleanup_text(text)
        txt.append(text)

        pix_per = round((pix / shape ) * 100,2)

    img_pix.append(shape)
    txt_pix.append(pix)

    txtt.append(txt)
    img_nme.append(img_name)
    pix_perc.append(pix_per)

    contr_r.append(contrast_r)
    height.append(img.shape[0])
    width.append(img.shape[1])
    dtf = pd.DataFrame(list(zip(img_nme, img_pix, txt_pix, pix_perc, txtt,
                                        contr_r, width, height )), 
                columns =['Image', 'Image_pixels', 'Text_pixels', 'text_perc', 'text', 
                           'contrast_r', 'width', 'height']) 

    def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
        text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

        ## Tokenize (convert from string to list)
        lst_text = text.split()
        ## remove Stopwords
        if lst_stopwords is not None:
            lst_text = [word for word in lst_text if word not in 
                        lst_stopwords]

        ## Stemming (remove -ing, -ly, ...)
        if flg_stemm == True:
            ps = nltk.stem.porter.PorterStemmer()
            lst_text = [ps.stem(word) for word in lst_text]

        ## Lemmatisation (convert the word into root word)
        if flg_lemm == True:
            lem = nltk.stem.wordnet.WordNetLemmatizer()
            lst_text = [lem.lemmatize(word) for word in lst_text]

        ## back to string from list
        text = " ".join(lst_text)
        return text

    dtf["text_clean"] = dtf["text"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords = None))

    dtf['word_count'] = dtf["text"].apply(lambda x: len(str(x).split(" ")))
    dtf['char_count'] = dtf["text"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
    dtf['sentence_count'] = dtf["text"].apply(lambda x: len(str(x).split(".")))
    dtf['avg_word_length'] = round(dtf['char_count'] / dtf['word_count'],0)
    dtf['avg_sentence_lenght'] = dtf['word_count'] / dtf['sentence_count']

    dtf1 = dtf[['Image', 'text_perc', 'word_count' , 'avg_word_length']]
    
    global df1
    df1 = dtf1
    print("code run successfully")

app.layout = html.Div([
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-image-upload'),
    
        html.Hr(),
    
 ])

def parse_contents(contents, filename, date):
    return html.Div([
        html.H6(filename),
        html.Img(src=contents, width = 500),
        html.Hr(),
        generate_table(df1),
        html.Hr(),
        html.H6('Recommended text percent : < 19'), #####Mention your own metrics which you need to control
        html.H6('Recommended word count : < 18'),
        html.H6('Average word length : 6 to 8')
          ])

@app.callback(Output('output-image-upload', 'children'),
              Input('upload-image', 'contents'),
              State('upload-image', 'filename'),
              State('upload-image', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        global aa
        aa = list_of_contents
        
        global bb
        bb = list_of_names
        img_conv()
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)
        ]
        return children

    
app.run_server(mode='external', port=8060)   ####Mention the server you need to run the code

Dash app running on http://127.0.0.1:8060/
